# 🐶End-to-end Multiclass Dog Breed Classification

This notebook builds an end-to-end multi class image classifier using Tensorflow 2.x and Tensorflow Hub.

## 1. Problem

Identify the breed of the dog by the given image of the dog.

When I'm sitting at the cafe and I take a photo of a dog, I want to know what breed of dog it is.


## 2. Data


The data we're using is from Kaggle's dog breed identification competition.

https://www.kaggle.com/c/dog-breed-identification/data


## 3. Evaluation

The evaluation is a file with prediction probabilities for each dog breed of each test image.

https://www.kaggle.com/c/dog-breed-identification/overview/evaluation


## 4. Features

Some features about data:
* We're are dealing with images (unstructured data) so it's probably best to use deep learning/ transfer learning.
* There are 120 different breeds of dogs (This means there are 120 different classes).
* There are 10,000+ images in the  training set (these images have labels).
* There are 10,000+ images in the test set (these images have no labels since we'll be predicting  




In [1]:
# Unzip the data
# !unzip "/content/drive/MyDrive/Colab Notebooks/Dog Vision /dog-breed-identification.zip" -d "/content/drive/MyDrive/Colab Notebooks/Dog Vision "

## Get our workspace ready

- Import Tensorflow 2.x ✅
- Import Tensorflow Hub ✅
- Make sure we're using a GPU ✅

In [2]:
# Import Tensorflow
import tensorflow as tf
import tensorflow_hub as hub
print("Tensorflow version:", tf.__version__)
print("Tensorflow_hub version:", hub.__version__)

# Check for GPU availability
print("GPU", "available (YES!)" if tf.config.list_physical_devices("GPU") else
      "not available :(")

AttributeError: module 'tensorflow._api.v2.compat.v2.__internal__' has no attribute 'register_load_context_function'

## Getting our data ready (turning into tensors)

With all machine learning models, our data  has to be in numerical format.So that's what we will be doing first.Turning our images into Tensors(numerical representation).

Let's start by accessing our data and checking out the labels.

In [ ]:
# check out the label of the data

import pandas as pd
labels_csv = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/Dog Vision/labels.csv")
labels_csv.head()

In [ ]:
labels_csv.describe()

In [ ]:
# how many images are there in each breed?
labels_csv["breed"].value_counts().plot.bar(figsize=(20,10));

In [ ]:
# What's the median number of images per class?
labels_csv["breed"].value_counts().median()

In [ ]:
# lets view an image
from IPython.display import Image
Image("/content/drive/MyDrive/Colab Notebooks/Dog Vision/train/000bec180eb18c7604dcecc8fe0dba07.jpg")

### Getting Images and their labels

Let's get a list of all our images file pathnames.

In [ ]:
labels_csv.head()

In [ ]:
# Creat a path name for image Id's

filenames = ["drive/MyDrive/Colab Notebooks/Dog Vision/train/" + fname + ".jpg" for fname in labels_csv[ "id"]]

# Check for forst 10 rows
filenames[:10]

In [ ]:
# Check whether number of filenames matches number of actual image files

import os
if len(os.listdir("/content/drive/MyDrive/Colab Notebooks/Dog Vision/train")) == len(filenames):
  print("Filename matches actual number of files ! Proceed.")
else:
  print("File name not matched check target directory")


In [ ]:
# one miore check
Image(filenames[1000])

In [ ]:
labels_csv["breed"][1000]

Since we got our training image filepaths in a list,let's prepare our list.

In [ ]:
import numpy as np
labels = labels_csv["breed"]
labels = np.array(labels)
labels

In [ ]:
len(labels)

In [ ]:
# See if the number of labels matches the number of filenames
if len(labels) == len(filenames):
  print("The number of labels matches the number of filenames")
else:
  print("Number of labels does not matches the number of filenames, check the data directories")

In [ ]:
# Find the unique label values
unique_breeds = np.unique(labels)
unique_breeds

In [ ]:
# Turn a single label into am array of booleans
print(labels[0])
labels[0] == unique_breeds


In [ ]:
len(labels)

In [ ]:
# Turn every label into boolean array
boolean_labels = [label == unique_breeds for label in labels]
boolean_labels[:2]

In [ ]:
len(boolean_labels)

In [ ]:
# Example: Turning boolean numbers into integers
print(labels[0]) # orginal label
print(np.where(unique_breeds==labels[0])) # index where the label occurs
print(boolean_labels[0].argmax()) # index where the label occurs in boolean array
print(boolean_labels[0].astype(int)) # there will be 1 where the sample label occurs

In [ ]:
print(labels[1])
print(boolean_labels[1].astype(int))

In [ ]:
boolean_labels[:2]

In [ ]:
filenames[:10]

# Creating our own validation set
Since the dataset from kaggle does'nt come with validation set,we're going to create our own.

In [ ]:
# setup x and y variables
x = filenames # our data (images)
y = boolean_labels # labels of the images in boolean

In [ ]:
len(filenames)

We're going to strt off with ~1000 images and increase as needed.

In [ ]:
# Set number of images to use for experimenting
NUM_IMAGES = 1000 #@param {type:"slider",min:1000,max:10000,step:1000}

In [ ]:
# let's split our data into train and validation sets
from sklearn.model_selection import train_test_split

# split the data into train and validation set of total NUM_IMAGES size
x_train, x_val, y_train, y_val = train_test_split(x[:NUM_IMAGES],
                                                  y[:NUM_IMAGES],
                                                  test_size=0.2,
                                                  random_state=42)

len(x_train), len(x_val), len(y_train), len(y_val)

In [ ]:
# let's check
x_train[:5],y_train[:2]

## Preprocessing Images (turning images into Tensors)

To Preprocess our images into Tensor's we're going to write a function that does a few things.

1. Take an image file as input.
2. Use TensorFlow to read the file and save it to a variable,`image`
3. Turn our `image` (a jpg) into tensors
4. Normalize our image (convert colour channel values from 0-255 to 0-1).
5. Resize the `image` to be a shape of (224,224).
6. Return the modified `image`.

Before we do,let's see how does an image look like when we import it.


In [ ]:
# Convert image into numpy array
from matplotlib.pyplot import imread
image = imread(filenames[20])
image.shape # return height,width,colour channel number(RGB)

In [ ]:
image[:2]

In [ ]:
# turn image into tensor
tf.constant(image)[:2]

Now we have seen what an image looks like as a Tensor,let's make a function to preprocess them.

We'll create a function to.


1. Take an image file as input.
2. Use TensorFlow to read the file and save it to a variable,`image`
3. Turn our `image` (a jpg) into tensors
4. Normalize our image (convert colour channel values from 0-255 to 0-1).
5. Resize the `image` to be a shape of (224,224).
6. Return the modified `image`.

More information on loading images in TensorFlow can be seen here: https://www.tensorflow.org/tutorials/load_data/images


In [ ]:
# Define a image size
IMG_SIZE = 244

# Create a function for preprocessing images
def process_image(img_path,img_size = IMG_SIZE):
  """
  Take an image file path and converts the image into Tensor
  """

  # Read in an image file
  image = tf.io.read_file(img_path)
  # Turn the jpg image into Numerical Tensor with 3 colour channels (Red,Green,Blue)
  image = tf.image.decode_jpeg(image, channels=3)
  # Convert colour channel values from 0-255 to 0-1
  image = tf.image.convert_image_dtype(image, tf.float32)
  # Resize the image to desired value (224,224)
  image = tf.image.resize(image, size=[IMG_SIZE, IMG_SIZE])

  return image


## Turn our data into batches

why should we turn our data into batches?

Let's say you're trying to process 10,000+ images in one go.They all might not fit into memory(GPU shortage occur).

So that's why we do 32(batch size) images at a time (you can manually adjust the batch size if needed).

In order to use Tensorflow effectively, we need our data in the form of Tensor tuples which look like this:
`(image, label)`.

In [ ]:
# Create a simple function to return a tuple (image, label)
def get_image_label(image_path, label):
  """
  Takes an image file path name and the associated label,
  processes the image and returns a tuple of (image,label).
  """
  image = process_image(image_path)
  return image, label

In [ ]:
# Demo of the above
(process_image(x[42],  tf.constant(y[42])))

Now we have got a way to turn our data into tuples of Tensors in the form:`(image,label),let's
make a function to turn all of our data (`x` & `y`) into batches.

In [ ]:
# Define the batch size, 32 is a good start
BATCH_SIZE = 42

# Create a function to turn dat into batches
def create_data_batches(x,y=None, batch_size=BATCH_SIZE, valid_data=False, test_data=False):

  """
  Creates batches of data out of image (y) and label (y) pairs.
  Shuffles the data if it's training data and does'nt shuffle if it's validation data.
  Also accepts the data as input (no labels).
  """
  # If the data is a test dataset, we probably don't have any labels
  if test_data:
    print("Creating test data batches...")
    data = tf.data.Dataset.from_tensor_slices((tf.constant(x))) # only filepaths (no labels)
    data_batch = data.map(process_image).batch(BATCH_SIZE)
    return data_batch

  # If the data is valid dataset, we don't need to shuffle it
  elif valid_data:
    print("Create validation data batches...")
    data = tf.data.Dataset.from_tensor_slices((tf.constant(x), # filepaths
                                               tf.constant(y))) #labels
    data_batch = data.map(process_image).batch(BATCH_SIZE)
    return data_batch

  else:
    print("Creating training data batches...")
    # Turn filepaths and labels into Tensors
    data = tf.data.Dataset.from_tensor_slices((tf.constant(x),
                                               tf.constant(y)))
    # Shuffling pathnames and labels before mapping the image processor function is faster than shuffling images
    data = data.shuffle(buffer_size=len(x))

    # Create (image,label) tuples (this also turns the image path into a preprocessed image)
    data = data.map(get_image_label)

    # Turn the training data into batches
    data_batch = data.batch(BATCH_SIZE)

  return data_batch


In [ ]:
# Create training and validation batches
train_data = create_data_batches(x_train,y_train)
val_data = create_data_batches(x_val,y_val,valid_data=True)

In [ ]:
# Checkout the differnt attributes of our data batches
train_data.element_spec, val_data.element_spec

## Visualizing Data Batches

Our data is now in batches, however these can be a little hard to understand/comprehend,let's visulize them!

In [ ]:
import matplotlib.pyplot as plt

# Create a function for viewing images in the data batch
def show_25_images(images, labels):
  """
  Displays a plot of 25 images and their labels from a data batch
  """
  # Setup the figure
  plt.figure(figsize=(10, 10))
  # Loop through 25 (for displaying 25 images)
  for i in range(25):
    # Create subplots(5 rows, 5 columns)
    ax = plt.subplot(5, 5, i+1)
    # Display an image
    plt.imshow(images[i])
    # Add the image label as the title
    plt.title(unique_breeds[labels[i].argmax()])
    # Turn the grid off
    plt.axis("off")

In [ ]:
train_data

In [ ]:
# Now let's visualize the data in a training batch
train_images, train_labels = next(train_data.as_numpy_iterator())
show_25_images(train_images, train_labels)

In [ ]:
dataset = tf.data.Dataset.range(2)
for element in dataset:
  print(element)



In [ ]:
# # Now let's visualize the data in our validation batch
# val_images, val_labels = next(val_data.as_numpy_iterator())
# show_25_images(train_images, val_labels)

## Building a model

Before we build a model,there are a few things we need to define:

- The input shape (our image shape, in the form of Tensors) to our model.
- The output shape (image labels, in the form of Tensors ) of our model.
- The URL of the model we want to use from Tensorflow Hub - https://www.kaggle.com/models/google/mobilenet-v2/tensorFlow2/130-224-classification/1?tfhub-redirect=true

In [ ]:
IMG_SIZE

In [ ]:
# setup input shape to the model

# reinstantiate the IMG_SIZE

IMG_SIZE = 224

INPUT_SHAPE = [None, IMG_SIZE , IMG_SIZE, 3] # batch, height, width, colour channels

# Setup output shape of our model
OUTPUT_SHAPE = len(unique_breeds)

# Setup model URL form TensorFlow Hub
MODEL_URL = "https://www.kaggle.com/models/google/mobilenet-v2/tensorFlow2/130-224-classification/1?tfhub-redirect=true"

In [ ]:
INPUT_SHAPE

Now we've have got our inputs,outputs and model ready to go. Let's put them together into keras deep learning model.

Knowing this,let's create a function which:

- Takes the input shape,output shape and the model we've chosen as parameters.
- Defines the layers in a keras model in sequential fashion(do this first,then this,then that)
- Compiles the model (says it should be evaluated and improved).
- Builds the model(tells the model the input shape it'll be getting).
- Returns the model.

All of these steps can be found here: https://www.tensorflow.org/guide/keras/overview

In [ ]:
# Create a function which builds a keras model

def create_model(input_shape=INPUT_SHAPE, output_shape=OUTPUT_SHAPE, model_url=MODEL_URL):
  print("Building model with:",MODEL_URL)

  # Setup the model layers
  model = tf.keras.Sequential([
      hub.KerasLayer(MODEL_URL), # Layer 1 (input layer)
      tf.keras.layers.Dense(units=OUTPUT_SHAPE,
                            activation="softmax") # Layer 2 (output layer)
  ])

  # Compile the model
  model.compile(
      loss = tf.keras.losses.CategoricalCrossentropy(),
      optimizer=tf.keras.optimizers.Adam(),
      metrics=["accuracy"]
  )

  # Build the model
  model.build(INPUT_SHAPE)

  return model

In [ ]:
model = create_model()
model.summary()

## Creating callbacks

Callbacks are helper functions a model can use during to do such things as save its progress,check its progress or stop training early if
a model stops improving.

We'll create two callbacks, one for TensorBoard whcih helps track our models
progress and another for early stopping which prevents our model from training for too long.

### TensorBoard Callback

To setup a TensorBoard callback, we need to do 3 things:

1. Load the TensorBoard notebook extension ✅
2. Create a TensorBoard callback which is able to save logs to a directory and pass it to our model's `fit()` function. ✅
3. Visualize our models training logs with the `%tensorboard` magic function(we'll do this after model training).

https://www.tensorflow.org/api_docs/python/tf/keras/callbacks/TensorBoard

In [ ]:
# Load TensorBoard notebook extension

%load_ext tensorboard

In [ ]:
import datetime

# Create a function to build a TensorFlow callback
def create_tensorboard_callback():
  # Create a log directory for storing TensorBoard logs
  logdir = os.path.join("/content/drive/MyDrive/Colab Notebooks/Dog Vision/logs",
                        # Make it so the logs get tracked whenever we run an experiment
                        datetime.datetime.now().strftime("%d%m%Y-%H%M%S"))
  return tf.keras.callbacks.TensorBoard(logdir)


## Early Stopping Callback

Early stopping helps stop our model from overfitting by stopping training if a certain evaluation metric stops improving.

https://www.tensorflow.org/api_docs/python/tf/keras/callbacks/EarlyStopping


In [ ]:
# Create early stopping callback
early_stopping = tf.keras.callbacks.EarlyStopping(monitor="val_accuracy",
                                                  patience=3)


## Training a model (on subset of data)

Our first model is only going to train on 1000 images, to make sure everything is working.

In [ ]:
NUM_EPOCHS = 60 # @param {type:"slider", min:10, max:100, step:10 }

In [ ]:
# check to make sure we're still running on a GPU
print("GPU", "available(YES!!!!)" if tf.config.list_physical_devices("GPU") else "not available :(")

  Let's create a function which trains a model.

  - Create a model using `create_model()`
  - Setup a TensorBoard callback using `create_tensorboard_callback()`
  - Call the `fit()` function on our model passing it the training data,validation data, number of epochs to train for (`NUM_EPOCHS`) and the callbacks we'd like
  to use.
  - Return the model

In [ ]:
# Build a function to train and return a trained model
def train_model():
  """
  Trains a given model and returns the trained version
  """
  # Create a model
  model = create_model()

  # Create new TensorBoard session everytime we train a model
  tensorboard = create_tensorboard_callback()

  # Fit the model to the data passing it the callbacks we created
  model.fit(x=train_data,
            epochs=NUM_EPOCHS,
            validation_data = val_data,
            validation_freq = 1,
            callbacks = [tensorboard, early_stopping])
  # Return the fitted model
  return model

In [ ]:
# Fit the model to the data
model = train_model()